### Finetuning only the last layer
F1 with Rekall labels (1 seed, 25 epochs, 16 batch size):

* 0.8410 @ 20\% {'train': 13115, 'val_train': 9479, 'val': 9479, 'test': 7496}
* 0.8647 @ 40\% {'train': 25552, 'val_train': 9479, 'val': 9479, 'test': 7496}
* 0.8788 @ 100\% {'train': 64130, 'val_train': 9479, 'val': 9479, 'test': 7496}
* 0.xxxx @ 500\% {'train': 344594, 'val_train': 9479, 'val': 9479, 'test': 7496}

In [1]:
# hyperparameters
# with m2200 - 15min with 8, 10min with 16, 9min with 24 and out of memory with 32
data_root = '/home/rsundar/cs229/commercials_big/'
global_batch_size = 16 # originally set to 4, but we've agreed to use 16 as default

# parameters to control length of training
use_rekall_labels = True # False will use 'true' labels - files need to be in 'commercials\\data\\'
data_percentage = 5     # percentage of training data to us
num_seeds = 1            # number of different seeds to run, originally 5
global_num_epochs = 25   # originally 25

In [2]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import time
import os
import copy
from PIL import Image
from tqdm import tqdm
from scipy import stats

In [3]:
device = torch.device('cuda:0')

In [4]:
# data_transforms = {
#     'train': transforms.Compose([
# #         transforms.Resize((310, 310)),
#         transforms.Resize((224, 224)),
# #         transforms.CenterCrop(224),
# #         transforms.RandomResizedCrop(224),
# #         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
#     'val': transforms.Compose([
# #         transforms.Resize((310, 310)),
#         transforms.Resize((224, 224)),
# #         transforms.Resize(256),
# #         transforms.CenterCrop(224),
# #         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
# }
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [5]:
class ImageDataset(Dataset):
    def __init__(self, paths, labels, transform=None, stride = 1, max_size=None):
        self.paths = paths[::stride]
        if max_size is not None:
            self.paths = self.paths[:max_size]
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img_name = self.paths[idx]
        img = Image.open(img_name).convert('RGB')
        
        img_tensor = self.transform(img)

        return img_tensor, self.labels[idx]

In [6]:
train_file = 'commercials/data/rekall_train.txt'
train_file_new_points = 'commercials/data/rekall_big_train_new.txt'
val_file = 'commercials/data/val.txt'
test_file = 'commercials/data/test.txt'

In [7]:
def read_file(filename, filename_new_points=None, load_percentage = 1):
    # Assumes that there is no overlap in datapoints between filename and filename_new_points!
    paths = []
    labels = []
    videos = []
    
    num_total_videos = 0
    
    with open(data_root + filename, 'r') as f:
        
        # find set of unique videos on file
        for line in f.readlines():
            video, image, label = line.split(' ')
            videos.append(video)
        videos = sorted(list(set(videos))) # set of all videos
        num_total_videos = len(videos)
        num_videos = int(len(videos) * load_percentage) # number of videos to use
        videos = videos[:num_videos] # take first 'num_videos'
        f.seek(0) # go back to beginning of file
        
        # read in data only using videos in possibly reduced set from above
        for line in f.readlines():
            video, image, label = line.split(' ')
            if video in videos: # load it up only if it's in the list of videos to use
                paths.append(data_root + 'commercials/images/{}/{:04d}.jpg'.format(video, int(image)))
                labels.append(int(label))
    
    videos = []
    if filename_new_points is not None and load_percentage > 1:
        print("Fetching points from new data file")
        with open(data_root + filename_new_points, 'r') as f:

            # find set of unique videos on file
            for line in f.readlines():
                video, image, label = line.split(' ')
                videos.append(video)
            videos = sorted(list(set(videos))) # set of all videos
            num_videos_to_fetch = int(num_total_videos * (load_percentage - 1))
            if num_videos_to_fetch > len(videos):
                print("Trying to fetch a larger data_percentage than is available")
                num_videos_to_fetch = len(videos)
            videos = videos[:num_videos_to_fetch] # take first 'num_videos'
            f.seek(0) # go back to beginning of file

            # read in data only using videos in possibly reduced set from above
            for line in f.readlines():
                video, image, label = line.split(' ')
                if video in videos: # load it up only if it's in the list of videos to use
                    paths.append(data_root + 'commercials/images/{}/{:04d}.jpg'.format(video, int(image)))
                    labels.append(int(label))
            
    return paths, labels

In [8]:
train_paths, Y_train = read_file(train_file, train_file_new_points, load_percentage=data_percentage)
val_paths  , Y_val   = read_file(val_file)
test_paths , Y_test  = read_file(test_file)

Fetching points from new data file


In [9]:
# val_train is the same as val
image_datasets = {
    'train'    : ImageDataset(train_paths, Y_train, transform = data_transforms['train'], stride=1, max_size=None),
    'val_train': ImageDataset(val_paths  , Y_val  , transform = data_transforms['train'], stride=1, max_size=None),
    'val'      : ImageDataset(val_paths  , Y_val  , transform = data_transforms['val']  , stride=1, max_size=None),
    'test'     : ImageDataset(test_paths , Y_test , transform = data_transforms['val']  , stride=1, max_size=None)
}

In [10]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                              batch_size = global_batch_size,
                                              shuffle = 'train' in x,
                                              num_workers = 4,
                                              pin_memory = True)
                                              for x in image_datasets}

In [11]:
dataset_sizes = {
    x: len(image_datasets[x])
    for x in image_datasets
}

In [12]:
dataset_sizes

{'train': 344594, 'val_train': 9479, 'val': 9479, 'test': 7496}

In [13]:
def safe_divide(a, b):
    return a / b if b > 0 else 0

In [14]:
def train_model(model, criterion, optimizer, scheduler, train_dl, val_dl, test_dl=None,
                num_epochs=25, return_best=False, verbose=True, log_file=None, path=None):
    print(train_dl, val_dl, test_dl)
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_epoch = 0
    best_test_acc = 0.0
    best_f1 = 0.0
    best_precision = 0.0
    best_recall = 0.0
    
    phases = ['train', 'val', 'test'] if test_dl is not None else ['train', 'val']

    for epoch in range(num_epochs):
        t_start = time.time()
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in phases:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
                dl = dataloaders[train_dl]
                dataset_size = dataset_sizes[train_dl]
                
            elif phase == 'val':
                model.eval()   # Set model to evaluate mode
                dl = dataloaders[val_dl]
                dataset_size = dataset_sizes[val_dl]
            else:
                model.eval()
                dl = dataloaders[test_dl]
                dataset_size = dataset_sizes[test_dl]

            running_loss = 0.0
            running_corrects = 0
            true_positives = 0.
            true_negatives = 0.
            false_positives = 0.
            false_negatives = 0.

            # Iterate over data.
            for inputs, labels in dl:
                inputs = inputs.to(device)
                labels = labels.to(device).float()

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    preds = torch.where(
                        outputs >= 0.,
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                    target = torch.where(
                        labels >= 0.5,
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device)
                    )
                    loss = criterion(outputs.view(target.shape), target)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                label_vals = torch.where(
                    labels >= 0.5,
                    torch.tensor([1.]).to(device),
                    torch.tensor([0.]).to(device)
                )
                correct = preds.view(label_vals.shape) == label_vals.data
                running_corrects += torch.sum(correct)
                
                true_positives += torch.sum(
                    torch.where(
                        (correct == 1.) * (label_vals == 1.),
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                )
                true_negatives += torch.sum(
                    torch.where(
                        (correct == 1.) * (label_vals == 0.),
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                )
                false_positives += torch.sum(
                    torch.where(
                        (correct == 0.) * (label_vals == 0.),
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                )
                false_negatives += torch.sum(
                    torch.where(
                        (correct == 0.) * (label_vals == 1.),
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                )
            
            epoch_loss = running_loss / dataset_size
            epoch_acc = running_corrects.double() / dataset_size
            epoch_pre = safe_divide(true_positives, (true_positives + false_positives))
            epoch_recall = safe_divide(true_positives, (true_positives + false_negatives))
            epoch_f1 = safe_divide(2 * epoch_pre * epoch_recall, (epoch_pre + epoch_recall))

            if verbose:
                print('{} Loss: {:.4f} Acc: {:.4f} Pre: {:.4f} Rec: {:.4f} F1: {:.4f} Time in epoch: {:.4f}'.format(
                    phase, epoch_loss, epoch_acc, epoch_pre, epoch_recall, epoch_f1, time.time() - t_start))
                print('TP: {} TN: {} FP: {} FN: {}'.format(
                    true_positives.data, true_negatives.data, false_positives.data, false_negatives.data))
            if log_file is not None:
                log_file.write('Phase: {0}\t'
                               'Epoch: [{1}/{2}]\t'
                               'Loss: {loss_c:.4f}\t'
                               'Acc: {acc:.4f}\t'
                               'Pre: {pre:.4f}\t'
                               'Rec: {rec:.4f}\t'
                               'F1: {f1:.4f}\t'
                               'TP: {tp} '
                               'TN: {tn} '
                               'FP: {fp} '
                               'FN: {fn}\n'.format(
                                   phase, epoch + 1, num_epochs, loss_c=epoch_loss,
                                   acc=epoch_acc, pre=epoch_pre, rec=epoch_recall,
                                   f1=epoch_f1, tp=int(true_positives.data), tn=int(true_negatives.data),
                                   fp=int(false_positives.data), fn=int(false_negatives.data)
                               ))
                log_file.flush()
            if path is not None:
                torch.save(model.state_dict(), os.path.join(path, 'seed_{}_epoch_{}.pth'.format(seed, epoch)))

            # deep copy the model
            if phase == 'val' and epoch_f1 > best_f1:
                best_acc = epoch_acc
                best_f1 = epoch_f1
                best_precision = epoch_pre
                best_recall = epoch_recall
                best_epoch = epoch
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'test' and best_epoch == epoch:
                best_test_acc = epoch_acc

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    
    if return_best:
        print('Best epoch: {}'.format(best_epoch))
        print('Best val Acc: {:4f}'.format(best_acc))
        print('Best val Pre: {:4f}'.format(best_precision))
        print('Best val Rec: {:4f}'.format(best_recall))
        print('Best val F1: {:4f}'.format(best_f1))
        print('Test Acc: {:4f}'.format(best_test_acc))

        # load best model weights
        model.load_state_dict(best_model_wts)
    return model

In [15]:
def get_frozen_model():
    # Useful links: 
    #   * https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html
    #   * https://stackoverflow.com/questions/42480111/model-summary-in-pytorch
    #   * https://discuss.pytorch.org/t/how-the-pytorch-freeze-network-in-some-layers-only-the-rest-of-the-training/7088/2
    model_ts = models.resnet50(pretrained=True)
    num_ftrs = model_ts.fc.in_features
    
    for param in model_ts.parameters():
        param.requires_grad = False
    model_ts.fc = nn.Linear(num_ftrs, 1)
    return model_ts

def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [16]:
torch.manual_seed(0)
model = get_frozen_model()
for param in model.parameters():
    if param.requires_grad:
        print(param, param.shape)
print(get_n_params(model))

Parameter containing:
tensor([[ 6.6970e-03, -6.0678e-03,  1.8278e-02,  ...,  1.2842e-02,
         -2.0258e-02,  9.5235e-05]], requires_grad=True) torch.Size([1, 2048])
Parameter containing:
tensor([-0.0207], requires_grad=True) torch.Size([1])
23510081


In [ ]:
path = 'models/transfer_learning_b16_rekall_train_last_layer_500/'
for seed in range(1):
    torch.manual_seed(seed)
    # model_ts = models.resnet50(pretrained=True)
    # num_ftrs = model_ts.fc.in_features
    # model_ts.fc = nn.Linear(num_ftrs, 1)
    model_ts = get_frozen_model()

    model_ts = model_ts.to(device)

#     criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([10.]).to(device))
#     criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([5.]).to(device))
    
    criterion = nn.BCEWithLogitsLoss()
    
    # Observe that all parameters are being optimized
    optimizer_ts = optim.SGD(model_ts.parameters(), lr=0.001, momentum=0.9)

    # Decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler_ts = lr_scheduler.StepLR(optimizer_ts, step_size=7, gamma=0.1)
    
    if not os.path.exists(path):
        os.makedirs(path)
    with open(os.path.join(path, 'seed_{}.log'.format(seed)), 'w') as log_file:
        model_ts = train_model(model_ts, criterion, optimizer_ts, exp_lr_scheduler_ts,
                               'train', 'val', test_dl='test', num_epochs=25, verbose=True,
                               log_file=log_file, return_best=False, path=path)
        # torch.save(model_ts.state_dict(), os.path.join(path, 'seed_{}.pth'.format(seed)))

train val test
Epoch 0/24
----------


/home/rsundar/miniconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


train Loss: 0.3721 Acc: 0.8347 Pre: 0.7311 Rec: 0.6439 F1: 0.6847 Time in epoch: 3955.3333
TP: 61859.0 TN: 225774.0 FP: 22757.0 FN: 34204.0
val Loss: 0.2368 Acc: 0.9003 Pre: 0.9490 Rec: 0.6947 F1: 0.8022 Time in epoch: 4044.7881
TP: 1916.0 TN: 6618.0 FP: 103.0 FN: 842.0
test Loss: 0.2728 Acc: 0.8778 Pre: 0.9318 Rec: 0.6622 F1: 0.7742 Time in epoch: 4114.5748
TP: 1570.0 TN: 5010.0 FP: 115.0 FN: 801.0

Epoch 1/24
----------
train Loss: 0.3564 Acc: 0.8451 Pre: 0.7450 Rec: 0.6758 F1: 0.7087 Time in epoch: 3759.3200
TP: 64924.0 TN: 226303.0 FP: 22228.0 FN: 31139.0
val Loss: 0.1957 Acc: 0.9358 Pre: 0.8779 Rec: 0.9050 F1: 0.8913 Time in epoch: 3844.3713
TP: 2496.0 TN: 6374.0 FP: 347.0 FN: 262.0
test Loss: 0.2102 Acc: 0.9253 Pre: 0.8832 Rec: 0.8802 F1: 0.8817 Time in epoch: 3911.5451
TP: 2087.0 TN: 4849.0 FP: 276.0 FN: 284.0

Epoch 2/24
----------
train Loss: 0.3522 Acc: 0.8476 Pre: 0.7491 Rec: 0.6818 F1: 0.7139 Time in epoch: 3776.3461
TP: 65499.0 TN: 226589.0 FP: 21942.0 FN: 30564.0
val Loss

In [39]:
path = 'models/transfer_learning_b16_rekall_train_last_layer_100/'
for seed in range(1):
    torch.manual_seed(seed)
    # model_ts = models.resnet50(pretrained=True)
    # num_ftrs = model_ts.fc.in_features
    # model_ts.fc = nn.Linear(num_ftrs, 1)
    model_ts = get_frozen_model()

    model_ts = model_ts.to(device)

#     criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([10.]).to(device))
#     criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([5.]).to(device))
    
    criterion = nn.BCEWithLogitsLoss()
    
    # Observe that all parameters are being optimized
    optimizer_ts = optim.SGD(model_ts.parameters(), lr=0.001, momentum=0.9)

    # Decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler_ts = lr_scheduler.StepLR(optimizer_ts, step_size=7, gamma=0.1)
    
    if not os.path.exists(path):
        os.makedirs(path)
    with open(os.path.join(path, 'seed_{}.log'.format(seed)), 'w') as log_file:
        model_ts = train_model(model_ts, criterion, optimizer_ts, exp_lr_scheduler_ts,
                               'train', 'val', test_dl='test', num_epochs=25, verbose=True,
                               log_file=log_file, return_best=False, path=path)
        # torch.save(model_ts.state_dict(), os.path.join(path, 'seed_{}.pth'.format(seed)))

train val test
Epoch 0/24
----------
train Loss: 0.3458 Acc: 0.8558 Pre: 0.7837 Rec: 0.7014 F1: 0.7403 Time in epoch: 580.2464
TP: 13178.0 TN: 41706.0 FP: 3637.0 FN: 5609.0
val Loss: 0.2278 Acc: 0.9142 Pre: 0.8505 Rec: 0.8557 F1: 0.8531 Time in epoch: 659.5008
TP: 2360.0 TN: 6306.0 FP: 415.0 FN: 398.0
test Loss: 0.2241 Acc: 0.9177 Pre: 0.8994 Rec: 0.8330 F1: 0.8649 Time in epoch: 722.8421
TP: 1975.0 TN: 4904.0 FP: 221.0 FN: 396.0

Epoch 1/24
----------
train Loss: 0.3197 Acc: 0.8692 Pre: 0.7981 Rec: 0.7412 F1: 0.7686 Time in epoch: 508.4394
TP: 13924.0 TN: 41820.0 FP: 3523.0 FN: 4863.0
val Loss: 0.2058 Acc: 0.9249 Pre: 0.8855 Rec: 0.8521 F1: 0.8684 Time in epoch: 588.0026
TP: 2350.0 TN: 6417.0 FP: 304.0 FN: 408.0
test Loss: 0.2149 Acc: 0.9192 Pre: 0.9099 Rec: 0.8262 F1: 0.8660 Time in epoch: 651.7489
TP: 1959.0 TN: 4931.0 FP: 194.0 FN: 412.0

Epoch 2/24
----------
train Loss: 0.3128 Acc: 0.8735 Pre: 0.8056 Rec: 0.7487 F1: 0.7761 Time in epoch: 508.1709
TP: 14066.0 TN: 41949.0 FP: 3394.

test Loss: 0.2061 Acc: 0.9221 Pre: 0.9146 Rec: 0.8313 F1: 0.8710 Time in epoch: 652.6959
TP: 1971.0 TN: 4941.0 FP: 184.0 FN: 400.0

Epoch 20/24
----------
train Loss: 0.2852 Acc: 0.8903 Pre: 0.8309 Rec: 0.7853 F1: 0.8075 Time in epoch: 509.3391
TP: 14754.0 TN: 42340.0 FP: 3003.0 FN: 4033.0
val Loss: 0.1853 Acc: 0.9356 Pre: 0.9022 Rec: 0.8735 F1: 0.8876 Time in epoch: 589.0807
TP: 2409.0 TN: 6460.0 FP: 261.0 FN: 349.0
test Loss: 0.2015 Acc: 0.9254 Pre: 0.9148 Rec: 0.8427 F1: 0.8773 Time in epoch: 652.8833
TP: 1998.0 TN: 4939.0 FP: 186.0 FN: 373.0

Epoch 21/24
----------
train Loss: 0.2864 Acc: 0.8893 Pre: 0.8298 Rec: 0.7825 F1: 0.8055 Time in epoch: 509.2605
TP: 14701.0 TN: 42328.0 FP: 3015.0 FN: 4086.0
val Loss: 0.1891 Acc: 0.9316 Pre: 0.9086 Rec: 0.8506 F1: 0.8787 Time in epoch: 589.1116
TP: 2346.0 TN: 6485.0 FP: 236.0 FN: 412.0
test Loss: 0.2055 Acc: 0.9206 Pre: 0.9181 Rec: 0.8224 F1: 0.8676 Time in epoch: 652.6662
TP: 1950.0 TN: 4951.0 FP: 174.0 FN: 421.0

Epoch 22/24
----------
tra

In [28]:
path = 'models/transfer_learning_b16_rekall_train_last_layer_40/'
for seed in range(1):
    torch.manual_seed(seed)
    # model_ts = models.resnet50(pretrained=True)
    # num_ftrs = model_ts.fc.in_features
    # model_ts.fc = nn.Linear(num_ftrs, 1)
    model_ts = get_frozen_model()

    model_ts = model_ts.to(device)

#     criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([10.]).to(device))
#     criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([5.]).to(device))
    
    criterion = nn.BCEWithLogitsLoss()
    
    # Observe that all parameters are being optimized
    optimizer_ts = optim.SGD(model_ts.parameters(), lr=0.001, momentum=0.9)

    # Decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler_ts = lr_scheduler.StepLR(optimizer_ts, step_size=7, gamma=0.1)
    
    if not os.path.exists(path):
        os.makedirs(path)
    with open(os.path.join(path, 'seed_{}.log'.format(seed)), 'w') as log_file:
        model_ts = train_model(model_ts, criterion, optimizer_ts, exp_lr_scheduler_ts,
                               'train', 'val', test_dl='test', num_epochs=25, verbose=True,
                               log_file=log_file, return_best=False, path=path)
        # torch.save(model_ts.state_dict(), os.path.join(path, 'seed_{}.pth'.format(seed)))

train val test
Epoch 0/24
----------
train Loss: 0.3751 Acc: 0.8365 Pre: 0.7662 Rec: 0.6658 F1: 0.7125 Time in epoch: 205.8512
TP: 5175.0 TN: 16200.0 FP: 1579.0 FN: 2598.0
val Loss: 0.2419 Acc: 0.9095 Pre: 0.9123 Rec: 0.7621 F1: 0.8305 Time in epoch: 284.4778
TP: 2102.0 TN: 6519.0 FP: 202.0 FN: 656.0
test Loss: 0.2656 Acc: 0.8925 Pre: 0.9187 Rec: 0.7242 F1: 0.8099 Time in epoch: 347.3604
TP: 1717.0 TN: 4973.0 FP: 152.0 FN: 654.0

Epoch 1/24
----------
train Loss: 0.3402 Acc: 0.8574 Pre: 0.7872 Rec: 0.7282 F1: 0.7565 Time in epoch: 202.9342
TP: 5660.0 TN: 16249.0 FP: 1530.0 FN: 2113.0
val Loss: 0.2281 Acc: 0.9161 Pre: 0.8532 Rec: 0.8597 F1: 0.8564 Time in epoch: 281.6697
TP: 2371.0 TN: 6313.0 FP: 408.0 FN: 387.0
test Loss: 0.2321 Acc: 0.9144 Pre: 0.8889 Rec: 0.8334 F1: 0.8603 Time in epoch: 344.5407
TP: 1976.0 TN: 4878.0 FP: 247.0 FN: 395.0

Epoch 2/24
----------
train Loss: 0.3219 Acc: 0.8682 Pre: 0.8052 Rec: 0.7475 F1: 0.7752 Time in epoch: 202.9447
TP: 5810.0 TN: 16373.0 FP: 1406.0 F

test Loss: 0.2161 Acc: 0.9185 Pre: 0.9044 Rec: 0.8300 F1: 0.8656 Time in epoch: 345.8309
TP: 1968.0 TN: 4917.0 FP: 208.0 FN: 403.0

Epoch 20/24
----------
train Loss: 0.2947 Acc: 0.8845 Pre: 0.8337 Rec: 0.7747 F1: 0.8031 Time in epoch: 202.9522
TP: 6022.0 TN: 16578.0 FP: 1201.0 FN: 1751.0
val Loss: 0.2068 Acc: 0.9265 Pre: 0.8730 Rec: 0.8745 F1: 0.8738 Time in epoch: 282.3615
TP: 2412.0 TN: 6370.0 FP: 351.0 FN: 346.0
test Loss: 0.2143 Acc: 0.9200 Pre: 0.8948 Rec: 0.8465 F1: 0.8700 Time in epoch: 345.9156
TP: 2007.0 TN: 4889.0 FP: 236.0 FN: 364.0

Epoch 21/24
----------
train Loss: 0.2970 Acc: 0.8815 Pre: 0.8295 Rec: 0.7682 F1: 0.7977 Time in epoch: 203.3916
TP: 5971.0 TN: 16552.0 FP: 1227.0 FN: 1802.0
val Loss: 0.2100 Acc: 0.9243 Pre: 0.8539 Rec: 0.8923 F1: 0.8727 Time in epoch: 283.0435
TP: 2461.0 TN: 6300.0 FP: 421.0 FN: 297.0
test Loss: 0.2134 Acc: 0.9225 Pre: 0.8881 Rec: 0.8638 F1: 0.8758 Time in epoch: 346.7855
TP: 2048.0 TN: 4867.0 FP: 258.0 FN: 323.0

Epoch 22/24
----------
train

In [17]:
path = 'models/transfer_learning_b16_rekall_train_last_layer_20/'
for seed in range(1):
    torch.manual_seed(seed)
    # model_ts = models.resnet50(pretrained=True)
    # num_ftrs = model_ts.fc.in_features
    # model_ts.fc = nn.Linear(num_ftrs, 1)
    model_ts = get_frozen_model()

    model_ts = model_ts.to(device)

#     criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([10.]).to(device))
#     criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([5.]).to(device))
    
    criterion = nn.BCEWithLogitsLoss()
    
    # Observe that all parameters are being optimized
    optimizer_ts = optim.SGD(model_ts.parameters(), lr=0.001, momentum=0.9)

    # Decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler_ts = lr_scheduler.StepLR(optimizer_ts, step_size=7, gamma=0.1)
    
    if not os.path.exists(path):
        os.makedirs(path)
    with open(os.path.join(path, 'seed_{}.log'.format(seed)), 'w') as log_file:
        model_ts = train_model(model_ts, criterion, optimizer_ts, exp_lr_scheduler_ts,
                               'train', 'val', test_dl='test', num_epochs=25, verbose=True,
                               log_file=log_file, return_best=False, path=path)
        # torch.save(model_ts.state_dict(), os.path.join(path, 'seed_{}.pth'.format(seed)))

train val test
Epoch 0/24
----------


/home/rsundar/miniconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


train Loss: 0.4016 Acc: 0.8237 Pre: 0.7560 Rec: 0.6251 F1: 0.6843 Time in epoch: 115.3591
TP: 2506.0 TN: 8297.0 FP: 809.0 FN: 1503.0
val Loss: 0.2578 Acc: 0.9045 Pre: 0.8802 Rec: 0.7777 F1: 0.8258 Time in epoch: 194.9282
TP: 2145.0 TN: 6429.0 FP: 292.0 FN: 613.0
test Loss: 0.2722 Acc: 0.8922 Pre: 0.8993 Rec: 0.7423 F1: 0.8133 Time in epoch: 258.4828
TP: 1760.0 TN: 4928.0 FP: 197.0 FN: 611.0

Epoch 1/24
----------
train Loss: 0.3543 Acc: 0.8515 Pre: 0.7824 Rec: 0.7121 F1: 0.7456 Time in epoch: 104.8160
TP: 2855.0 TN: 8312.0 FP: 794.0 FN: 1154.0
val Loss: 0.2505 Acc: 0.9049 Pre: 0.8500 Rec: 0.8176 F1: 0.8335 Time in epoch: 183.9567
TP: 2255.0 TN: 6323.0 FP: 398.0 FN: 503.0
test Loss: 0.2563 Acc: 0.8999 Pre: 0.8850 Rec: 0.7857 F1: 0.8324 Time in epoch: 247.0571
TP: 1863.0 TN: 4883.0 FP: 242.0 FN: 508.0

Epoch 2/24
----------
train Loss: 0.3507 Acc: 0.8512 Pre: 0.7782 Rec: 0.7176 F1: 0.7467 Time in epoch: 104.5226
TP: 2877.0 TN: 8286.0 FP: 820.0 FN: 1132.0
val Loss: 0.2416 Acc: 0.9124 Pre:


Epoch 20/24
----------
train Loss: 0.3194 Acc: 0.8691 Pre: 0.8037 Rec: 0.7565 F1: 0.7794 Time in epoch: 104.1547
TP: 3033.0 TN: 8365.0 FP: 741.0 FN: 976.0
val Loss: 0.2197 Acc: 0.9182 Pre: 0.8850 Rec: 0.8263 F1: 0.8547 Time in epoch: 182.8159
TP: 2279.0 TN: 6425.0 FP: 296.0 FN: 479.0
test Loss: 0.2351 Acc: 0.9072 Pre: 0.9001 Rec: 0.7946 F1: 0.8441 Time in epoch: 245.6677
TP: 1884.0 TN: 4916.0 FP: 209.0 FN: 487.0

Epoch 21/24
----------
train Loss: 0.3116 Acc: 0.8771 Pre: 0.8173 Rec: 0.7700 F1: 0.7930 Time in epoch: 103.9824
TP: 3087.0 TN: 8416.0 FP: 690.0 FN: 922.0
val Loss: 0.2332 Acc: 0.9175 Pre: 0.8452 Rec: 0.8771 F1: 0.8609 Time in epoch: 182.5882
TP: 2419.0 TN: 6278.0 FP: 443.0 FN: 339.0
test Loss: 0.2324 Acc: 0.9150 Pre: 0.8799 Rec: 0.8469 F1: 0.8631 Time in epoch: 245.4208
TP: 2008.0 TN: 4851.0 FP: 274.0 FN: 363.0

Epoch 22/24
----------
train Loss: 0.3204 Acc: 0.8721 Pre: 0.8127 Rec: 0.7556 F1: 0.7831 Time in epoch: 104.0362
TP: 3029.0 TN: 8408.0 FP: 698.0 FN: 980.0
val Loss: 

# Smooth Results

In [17]:
def pos_negs(predictions, gt):
    correct = np.where(np.array(predictions) == np.array(gt), 1, 0)
    incorrect = np.where(np.array(predictions) == np.array(gt), 0, 1)
    
    tp = np.where(correct * np.where(predictions == np.array(1), 1, 0), 1, 0)
    tn = np.where(correct * np.where(predictions == np.array(0), 1, 0), 1, 0)
    fp = np.where(incorrect * np.where(predictions == np.array(1), 1, 0), 1, 0)
    fn = np.where(incorrect * np.where(predictions == np.array(0), 1, 0), 1, 0)
    
    return tp, tn, fp, fn

def acc_prf1(predictions, gt):
    tp, tn, fp, fn = pos_negs(predictions, gt)
    
    acc = (np.sum(tp) + np.sum(tn)) / (np.sum(tp) + np.sum(tn) + np.sum(fp) + np.sum(fn))
    precision = np.sum(tp) / (np.sum(tp) + np.sum(fp))
    recall = np.sum(tp) / (np.sum(tp) + np.sum(fn))
    f1 = 2 * precision * recall / (precision + recall)
    
    return acc, precision, recall, f1, np.sum(tp), np.sum(tn), np.sum(fp), np.sum(fn)

def smooth_predictions(preds, window_radius = 3):
    result = []
    for i in range(len(preds)):
        start = max(0, i - window_radius)
        end = min(len(preds), i + window_radius)
        window = preds[start:end]
        result += [max(window, key=window.count)]
    
    return result

In [18]:
path = 'models/transfer_learning'

log_file_img = open(os.path.join(
    path, 'test_results_image_classifier.log'), 'w')
log_file_smoothed = open(os.path.join(
    path, 'test_results_smoothed.log'), 'w')

for seed in range(5):
    print(seed)
    model_ts = models.resnet50(pretrained=True)
    num_ftrs = model_ts.fc.in_features
    model_ts.fc = nn.Linear(num_ftrs, 1)
    model_ts.load_state_dict(torch.load(
        os.path.join(path, 'seed_{}.pth'.format(seed))
    ))
    model_ts.to(device)
    criterion = nn.BCEWithLogitsLoss()
    
    model = model_ts.eval()   # Set model to evaluate mode
    dl = dataloaders['test']
    dataset_size = dataset_sizes['test']
    
    running_corrects = 0
    true_positives = 0.
    true_negatives = 0.
    false_positives = 0.
    false_negatives = 0.

    i = 0

    predictions = []
    gt_labels = []
    # Iterate over data.
    for inputs, labels in dl:
        inputs = inputs.to(device)
        labels = labels.to(device).float()

        # forward
        # track history if only in train
        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            preds = torch.where(
                outputs >= 0.,
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device))
            target = torch.where(
                labels >= 0.5,
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device)
            )

        predictions += preds.cpu().numpy().tolist()
        gt_labels += labels.cpu().numpy().tolist()

        # statistics
        label_vals = torch.where(
            labels >= 0.5,
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device)
        )
        correct = preds.view(label_vals.shape) == label_vals.data
        running_corrects += torch.sum(correct)

        true_positives += torch.sum(
            torch.where(
                (correct == 1.) * (label_vals == 1.),
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device))
        )
        true_negatives += torch.sum(
            torch.where(
                (correct == 1.) * (label_vals == 0.),
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device))
        )
        false_positives += torch.sum(
            torch.where(
                (correct == 0.) * (label_vals == 0.),
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device))
        )
        false_negatives += torch.sum(
            torch.where(
                (correct == 0.) * (label_vals == 1.),
                torch.tensor([1.]).to(device),
                torch.tensor([0.]).to(device))
        )

        num_fp = torch.sum(
        torch.where(
            (correct == 0.) * (label_vals == 0.),
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device))
        )

    #     if num_fp > 0:
    #         print(num_fp)
    #         print(torch.where(
    #             (correct == 0.) * (label_vals == 0.),
    #             torch.tensor([1.]).to(device),
    #             torch.tensor([0.]).to(device)))
    #         print(i)
    #         out = torchvision.utils.make_grid(inputs)
    # #         imshow(out, title=preds.cpu().numpy().tolist())
    #         imshow(out)

        i += 1

    epoch_acc = running_corrects.double() / dataset_size
    epoch_pre = safe_divide(true_positives, (true_positives + false_positives))
    epoch_recall = safe_divide(true_positives, (true_positives + false_negatives))
    epoch_f1 = safe_divide(2 * epoch_pre * epoch_recall, (epoch_pre + epoch_recall))

    print('Acc: {:.4f} Pre: {:.4f} Rec: {:.4f} F1: {:.4f}'.format(
        epoch_acc, epoch_pre, epoch_recall, epoch_f1))
    print('TP: {} TN: {} FP: {} FN: {}'.format(
        true_positives.data, true_negatives.data, false_positives.data, 
        false_negatives.data))
    
    log_file_img.write('Seed: {0}\t'
                   'Acc: {acc:.4f}\t'
                   'Pre: {pre:.4f}\t'
                   'Rec: {rec:.4f}\t'
                   'F1: {f1:.4f}\t'
                   'TP: {tp} '
                   'TN: {tn} '
                   'FP: {fp} '
                   'FN: {fn}\n'.format(
                       seed,
                       acc=epoch_acc, pre=epoch_pre, rec=epoch_recall,
                       f1=epoch_f1, tp=int(true_positives.data),
                       tn=int(true_negatives.data),
                       fp=int(false_positives.data), fn=int(false_negatives.data)
                   ))
    log_file_img.flush()

    predictions = [p[0] for p in predictions]

    smoothed_preds = smooth_predictions(predictions, 3)

    print("Smoothed stats:")
    print(acc_prf1(smoothed_preds, gt_labels))
    
    sm_acc, sm_pre, sm_rec, sm_f1, sm_tp, sm_tn, sm_fp, sm_fn = acc_prf1(
        smoothed_preds, gt_labels)
    
    log_file_smoothed.write('Seed: {0}\t'
                   'Acc: {acc:.4f}\t'
                   'Pre: {pre:.4f}\t'
                   'Rec: {rec:.4f}\t'
                   'F1: {f1:.4f}\t'
                   'TP: {tp} '
                   'TN: {tn} '
                   'FP: {fp} '
                   'FN: {fn}\n'.format(
                       seed,
                       acc=sm_acc, pre=sm_pre, rec=sm_rec,
                       f1=sm_f1, tp=sm_tp,
                       tn=sm_tn,
                       fp=sm_fp, fn=sm_fn
                   ))
    log_file_smoothed.flush()

0
Acc: 0.9400 Pre: 0.8912 Rec: 0.9228 F1: 0.9068
TP: 2188.0 TN: 4858.0 FP: 267.0 FN: 183.0
Smoothed stats:
(0.9340981856990395, 0.8813490451036164, 0.9148038802193167, 0.8977649006622517, 2169, 4833, 292, 202)
1
Acc: 0.9506 Pre: 0.9216 Rec: 0.9224 F1: 0.9220
TP: 2187.0 TN: 4939.0 FP: 186.0 FN: 184.0
Smoothed stats:
(0.9450373532550693, 0.9145154464663563, 0.9114297764656263, 0.912970004224757, 2161, 4923, 202, 210)
2
Acc: 0.9452 Pre: 0.8911 Rec: 0.9418 F1: 0.9157
TP: 2233.0 TN: 4852.0 FP: 273.0 FN: 138.0
Smoothed stats:
(0.9389007470651014, 0.8883475436459602, 0.9228173766343315, 0.9052544476623914, 2188, 4850, 275, 183)
3
Acc: 0.9344 Pre: 0.8609 Rec: 0.9452 F1: 0.9011
TP: 2241.0 TN: 4763.0 FP: 362.0 FN: 130.0
Smoothed stats:
(0.9288954108858057, 0.8601097178683386, 0.9257697174188106, 0.8917326833231769, 2195, 4768, 357, 176)
4
Acc: 0.9338 Pre: 0.8692 Rec: 0.9308 F1: 0.8990
TP: 2207.0 TN: 4793.0 FP: 332.0 FN: 164.0
Smoothed stats:
(0.9303628601921025, 0.8678869876641464, 0.91986503584

# Understand Mistakes

In [29]:
path = 'models/transfer_learning'
model_ts = models.resnet50(pretrained=True)
num_ftrs = model_ts.fc.in_features
model_ts.fc = nn.Linear(num_ftrs, 1)
model_ts.load_state_dict(torch.load(
    os.path.join(path, 'seed_0.pth')
))
model_ts.to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([3.]).to(device))

In [30]:
model = model_ts.eval()   # Set model to evaluate mode
dl = dataloaders['test']
dataset_size = dataset_sizes['test']

In [31]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.cpu().numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(str(title))
    plt.pause(0.001)  # pause a bit so that plots are updated

In [32]:
def pos_negs(predictions, gt):
    correct = np.where(np.array(predictions) == np.array(gt), 1, 0)
    incorrect = np.where(np.array(predictions) == np.array(gt), 0, 1)
    
    tp = np.where(correct * np.where(predictions == np.array(1), 1, 0), 1, 0)
    tn = np.where(correct * np.where(predictions == np.array(0), 1, 0), 1, 0)
    fp = np.where(incorrect * np.where(predictions == np.array(1), 1, 0), 1, 0)
    fn = np.where(incorrect * np.where(predictions == np.array(0), 1, 0), 1, 0)
    
    return tp, tn, fp, fn

def acc_prf1(predictions, gt):
    tp, tn, fp, fn = pos_negs(predictions, gt)
    
    acc = (np.sum(tp) + np.sum(tn)) / (np.sum(tp) + np.sum(tn) + np.sum(fp) + np.sum(fn))
    precision = np.sum(tp) / (np.sum(tp) + np.sum(fp))
    recall = np.sum(tp) / (np.sum(tp) + np.sum(fn))
    f1 = 2 * precision * recall / (precision + recall)
    
    return acc, precision, recall, f1, np.sum(tp), np.sum(tn), np.sum(fp), np.sum(fn)

def smooth_predictions(preds, window_radius = 3):
    result = []
    for i in range(len(preds)):
        start = max(0, i - window_radius)
        end = min(len(preds), i + window_radius)
        window = preds[start:end]
        result += [max(window, key=window.count)]
    
    return result

In [33]:
running_corrects = 0
true_positives = 0.
true_negatives = 0.
false_positives = 0.
false_negatives = 0.

i = 0

predictions = []
gt_labels = []
# Iterate over data.
for inputs, labels in dl:
    inputs = inputs.to(device)
    labels = labels.to(device).float()

    # forward
    # track history if only in train
    with torch.set_grad_enabled(False):
        outputs = model(inputs)
        preds = torch.where(
            outputs >= 0.,
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device))
        target = torch.where(
            labels >= 0.5,
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device)
        )
        
    predictions += preds.cpu().numpy().tolist()
    gt_labels += labels.cpu().numpy().tolist()

    # statistics
    label_vals = torch.where(
        labels >= 0.5,
        torch.tensor([1.]).to(device),
        torch.tensor([0.]).to(device)
    )
    correct = preds.view(label_vals.shape) == label_vals.data
    running_corrects += torch.sum(correct)

    true_positives += torch.sum(
        torch.where(
            (correct == 1.) * (label_vals == 1.),
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device))
    )
    true_negatives += torch.sum(
        torch.where(
            (correct == 1.) * (label_vals == 0.),
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device))
    )
    false_positives += torch.sum(
        torch.where(
            (correct == 0.) * (label_vals == 0.),
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device))
    )
    false_negatives += torch.sum(
        torch.where(
            (correct == 0.) * (label_vals == 1.),
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device))
    )
    
    num_fp = torch.sum(
    torch.where(
        (correct == 0.) * (label_vals == 0.),
        torch.tensor([1.]).to(device),
        torch.tensor([0.]).to(device))
    )

#     if num_fp > 0:
#         print(num_fp)
#         print(torch.where(
#             (correct == 0.) * (label_vals == 0.),
#             torch.tensor([1.]).to(device),
#             torch.tensor([0.]).to(device)))
#         print(i)
#         out = torchvision.utils.make_grid(inputs)
# #         imshow(out, title=preds.cpu().numpy().tolist())
#         imshow(out)
    
    i += 1

epoch_acc = running_corrects.double() / dataset_size
epoch_pre = safe_divide(true_positives, (true_positives + false_positives))
epoch_recall = safe_divide(true_positives, (true_positives + false_negatives))
epoch_f1 = safe_divide(2 * epoch_pre * epoch_recall, (epoch_pre + epoch_recall))

print('Acc: {:.4f} Pre: {:.4f} Rec: {:.4f} F1: {:.4f}'.format(
    epoch_acc, epoch_pre, epoch_recall, epoch_f1))
print('TP: {} TN: {} FP: {} FN: {}'.format(
    true_positives.data, true_negatives.data, false_positives.data, false_negatives.data))

predictions = [p[0] for p in predictions]

smoothed_preds = smooth_predictions(predictions, 3)

print("Smoothed stats:")
print(acc_prf1(smoothed_preds, gt_labels))

Acc: 0.9421 Pre: 0.8904 Rec: 0.9317 F1: 0.9106
TP: 2209.0 TN: 4853.0 FP: 272.0 FN: 162.0
Smoothed stats:
(0.9327641408751334, 0.8783948115119579, 0.9139603542808942, 0.8958247209590741, 2167, 4825, 300, 204)
